In [294]:
function parse_map(input)
    H = length(input)
    W = length(input[1])

    map = zeros(Bool, (H,W))
    asteroids = []
    
    for j = 1:H
        for k = 1:W
            if input[j][k] == '#'
                map[j,k] = true
                push!(asteroids, (k,j))
            end
        end
    end
    return map, asteroids
end

function parse_map_X(input)
    H = length(input)
    W = length(input[1])
    println(H, W)
    map = zeros(Bool, (H,W))
    asteroids = []
    xpos = (0,0)
    
    for j = 1:H
        for k = 1:W
            if input[j][k] == '#'
                map[j,k] = true
                push!(asteroids, (k,j))
            end
            if input[j][k] == 'X'
                print("found")
                xpos = (k,j)
            end
        end
    end
    return map, asteroids, xpos
end

function count_map(asteroids, counts, map)
    cmap = zeros(Int, size(map))
    for (j,a) = enumerate(asteroids)
        cmap[a[2],a[1]] = counts[j]
    end
    return cmap
end

function count_visible(asteroids)
    N = length(asteroids)
    θs = []
    for j = 1:N
        xc, yc = asteroids[j]
        θ = []
        for a = asteroids[1:N .!= j]
            x,y = a
            push!(θ, atan(y-yc,x-xc))
        end
        push!(θs, θ)
    end
    
    return [length(Set(x)) for x in θs]
end

function get_best(asteroids, count)
    best = maximum(count)
    pos = asteroids[argmax(count)]
    return best, (pos[1]-1, pos[2]-1)
end

get_best (generic function with 1 method)

In [178]:
input = """.#..#
.....
#####
....#
...##"""
map, asteroids = parse_map(split(input, '\n'))
cnt = count_visible(asteroids)
count_map(asteroids, cnt, map)

5×5 Array{Int64,2}:
 0  7  0  0  7
 0  0  0  0  0
 6  7  7  7  5
 0  0  0  0  7
 0  0  0  8  7

In [179]:
input = """......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####"""
map, asteroids = parse_map(split(input, '\n'))
cnt = count_visible(asteroids)
count_map(asteroids, cnt, map)
get_best(asteroids, cnt)

(33, (5, 8))

In [180]:
input = """#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###."""
map, asteroids = parse_map(split(input, '\n'))
cnt = count_visible(asteroids)
count_map(asteroids, cnt, map)
get_best(asteroids, cnt)

(35, (1, 2))

In [181]:
input = """.#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#.."""
map, asteroids = parse_map(split(input, '\n'))
cnt = count_visible(asteroids)
count_map(asteroids, cnt, map)
get_best(asteroids, cnt)

(41, (6, 3))

In [320]:
convert(a) = (a[1]-1, a[2]-1)

convert (generic function with 1 method)

In [322]:
input = """.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##"""
map, asteroids = parse_map(split(input, '\n'))
cnt = count_visible(asteroids)
count_map(asteroids, cnt, map)
max, pos = get_best(asteroids, cnt)
print("Best $(max) at $(pos)")
c = (pos[1]+1, pos[2]+1)
v = vaporize(asteroids, c);

Best 210 at (11, 13)

In [327]:
v[200]

(8, 2)

In [328]:
input = readlines("./inputs/day10.txt")
map, asteroids = parse_map(input);
cnt = count_visible(asteroids)
count_map(asteroids, cnt, map)
best_visible, pos = get_best(asteroids, cnt)
c = (pos[1]+1, pos[2]+1)
v = vaporize(asteroids, c);

In [331]:
v[200][1]*100 + v[200][2]

806

In [321]:
struct Asteroid
    angle::Float64
    distance::Float64
    start::Tuple{Int,Int}
    pos::Tuple{Int,Int}
end

function Asteroid(a::Tuple{Int,Int}, c::Tuple{Int,Int}) 
    θ = mod2pi(pi/2 + atan(a[2]-c[2],a[1]-c[1]))
    d = sqrt((a[1]-c[1])^2 + (a[2]-c[2])^2)
    return Asteroid(θ, d, c, a)
end

function vaporize(asteroids, c)
    N = length(asteroids)
    aa = [Asteroid(a, c) for a in asteroids if a != c]
    sort!(aa)
    for a in aa
        #println(a)
    end
                
    used_angles = []
    vo = []
                
    for a in aa
        if a.angle in used_angles
            continue
        else
            push!(used_angles, a.angle)
            push!(vo, convert(a.pos))
        end
    end
    return vo
end

vaporize (generic function with 2 methods)

In [259]:
import Base: isless
function isless(a::Asteroid, b::Asteroid)
    if a.angle == b.angle
        return isless(a.distance, b.distance)
    else
        return isless(a.angle, b.angle)
    end
end

isless (generic function with 42 methods)

In [239]:
input = """
.#....#####...#..
##...##.#####..##
##...#...#.#####.
..#.....X...###..
..#.#.....#....##"""
map, asteroids,xpos = parse_map_X(split(input, '\n'))
vaporize(asteroids, xpos)

517
found

30-element Array{Any,1}:
 (9, 2) 
 (8, 1) 
 (7, 1) 
 (7, 2) 
 (6, 2) 
 (2, 1) 
 (6, 3) 
 (2, 2) 
 (1, 2) 
 (2, 3) 
 (1, 3) 
 (3, 4) 
 (3, 5) 
 ⋮      
 (16, 3)
 (15, 3)
 (14, 3)
 (13, 3)
 (16, 2)
 (12, 3)
 (13, 2)
 (12, 2)
 (10, 3)
 (11, 1)
 (10, 2)
 (10, 1)